In [1]:
import geopandas as gpd
from glob import glob
import dask.dataframe as da
from dask.distributed import LocalCluster, Client
import matplotlib.pyplot as plt
import os
import rioxarray as rio
import xarray as xr
# from dem_utils import ArcticDEM
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
from shapely import wkt
# import utils
import cartopy.crs as ccrs
import shapely
from rasterio.features import rasterize
from rasterio.enums import Resampling
import seaborn as sns
# import odc.geo.xr
prj = ccrs.Stereographic(
    central_latitude=90,
    central_longitude=-45,
    true_scale_latitude=70
)


In [11]:
dirs = glob('../data/id*')
for d in dirs:
    f = glob('elevation_sample*', root_dir=d)
    if len(f)==0:
        print(f'fuck: {d}')
    else:
        df = da.read_parquet(os.path.join(d, f[0]))
        print(len(df))

10210
12040
12513
10361
3093
9377
9554
9248
10347
10476
9154
14207
10709
14309
10337
8218
15506
14308
18675
10815
14296
20422
9514
16229
16856
15138
10947
7377
14816
10803
6528


In [ ]:
directory = '../data/id21_288157x_-930867y/'
sec_path = os.path.join(directory, 'sec.zarr')
dem_path = os.path.join(directory, 'stacked_coregd.zarr')
mask_path = os.path.join(directory, 'stable_terrain_mask.tif')

In [ ]:
def demote_coords_to_vars(ds: xr.Dataset,
                        coords: str,
                        var_name: str):
    '''
    messy onliner to for reorganizing dataset.
    e.g. dataset with two variables: a (dims: x, y, t) and b (dims: x, y)
    this function will convert it to a dataset with 
    dimensions x, y and add as many `a` variables as there dim `t` is long
    '''
    return xr.merge([
        ds.drop_vars([coords, var_name]),
        xr.merge(
            [ds[var_name].isel({coords:i}).rename(ds[coords][i].item())
            for i in range(len(ds[coords]))], compat='override').drop_vars(coords)]
                    )

def add_geom_mask(geom, buffer, ds):
    # buffer geometry, with square ends
    buff_geom = geom.buffer(200, cap_style=3)
    
    # empty array of same x, y dim shape as merged
    arr = np.zeros((ds.sizes['y'], ds.sizes['x']))
    
    # rasterize
    burned = rasterize(shapes=[(buff_geom, 1)],
                       fill=0,
                       out=arr,
                       transform=ds.rio.transform())
    
    # merged rasterized with all other dataarrays
    merged = xr.merge([ds, xr.DataArray(data=burned,
                                        dims=['y','x'],
                                        coords={'y': ds.y,
                                                'x': ds.x}).rename('buffer_aoi')])

    return merged


def get_summary_df(site):
    id = os.path.basename(site).split('_')[0][2:]
    
    cl_path = os.path.join(site, glob('*.geojson', root_dir=site)[0])
    cl = gpd.read_file(cl_path)
    where = cl.loc[0, 'where']
    lake_land = cl.loc[0, 'lake_land']
    
    sec_path = os.path.join(site, 'sec.zarr')
    dem_path = os.path.join(site, 'stacked_coregd.zarr')
    mask_path = os.path.join(site, 'stable_terrain_mask.tif')
    
    _sec = xr.open_zarr(sec_path)
    _sec = demote_coords_to_vars(_sec, 'result', 'sec')
    
    with xr.open_zarr(dem_path) as _dem:
        _dem_median = _dem['z'].median(dim='time',
                                       skipna=True).compute()
    
    _dem_median = _dem_median.rio.reproject_match(_sec,
                                            resampling=Resampling.bilinear,
                                            nodata=np.nan).rename('z_median')
    # _dem_median = reproject_like(_dem_median, _sec).rename('z_median').compute()
                
    with rio.open_rasterio(mask_path).squeeze().drop_vars('band') as _mask:
        mask_rprj = _mask.rio.reproject_match(_sec,
                                              resampling=Resampling.bilinear,
                                              nodata=0).rename('mask')

    # _dem_median_mask = xr.where(mask_rprj == 0, _dem_median, np.nan)
    # _sec_mask = xr.where(mask_rprj == 0, _sec, np.nan)
    merged = xr.merge([_dem_median, _sec, mask_rprj],
                      compat='override').compute()
       
    merged = add_geom_mask(cl.loc[0,'geometry'], 200, merged)
        
    ## take logical and of NOT stable terrain and centreline masks,
    cl_mask = ((merged['mask'] != 1) & (merged['buffer_aoi'] == 1))

    # set everything else to nan
    cl_masked = xr.where(cl_mask, merged, np.nan)

    # convert to dask dataframe
    df = (cl_masked
        .to_dask_dataframe()
        .dropna()
        .drop(columns='spatial_ref')
        .reset_index())

    # add additional meta
    df['where'] = where
    df['id'] = id
    df['lake_land'] = lake_land

    outpath = os.path.join(directory, 'elevation_sample.parquet')
    print(f'exporting to {outpath}')
    # return df, outpath
    da.to_parquet(df=df, path=outpath, compute=True)
    print('done')
    
    

In [ ]:
import os

In [8]:
pd.read_parquet('../data/id20_-225808x_-2493072y/elevation_sample.parquet/')

,index,y,x,z_median,n,slope,intercept,low_slope,high_slope,mask,buffer_aoi,where,id,lake_land
__null_dask_index__,,,,,,,,,,,,,,
0,311880,-2.492360e+06,-231300.0,294.565491,72.0,-1.679076,301.207842,-1.863758,-1.473365,-1.0,1.0,greenland,20,land
1,311881,-2.492360e+06,-231280.0,294.635681,72.0,-1.586579,300.842696,-1.834412,-1.369480,-1.0,1.0,greenland,20,land
2,311882,-2.492360e+06,-231260.0,294.965149,72.0,-1.699155,301.471282,-1.931409,-1.452984,-1.0,1.0,greenland,20,land
3,311883,-2.492360e+06,-231240.0,294.911652,73.0,-1.586162,300.851831,-1.799268,-1.383337,-1.0,1.0,greenland,20,land
4,311884,-2.492360e+06,-231220.0,295.426117,73.0,-1.487127,301.113956,-1.670903,-1.305947,-1.0,1.0,greenland,20,land
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15133,399539,-2.493702e+06,-217440.0,651.276855,91.0,-0.211007,652.174471,-0.403373,0.050258,-1.0,1.0,greenland,20,land
15134,399540,-2.493702e+06,-217420.0,651.298889,91.0,-0.212646,652.146090,-0.398980,0.040143,-1.0,1.0,greenland,20,land
15135,399541,-2.493702e+06,-217400.0,651.470581,91.0,-0.215888,652.352541,-0.402447,0.028028,-1.0,1.0,greenland,20,land


In [6]:
import pandas as pd
pd.read_parquet('../data/id21_288157x_-930867y/elevation_sample.parquet/')

,index,y,x,z_median,n,slope,intercept,low_slope,high_slope,mask,buffer_aoi,where,id,lake_land
__null_dask_index__,,,,,,,,,,,,,,
0,399120,-927885.050054,293849.709892,371.822632,27.0,-0.923722,377.357997,-0.990578,-0.878007,-1.0,1.0,greenland,21,land
1,400322,-927905.065288,293829.701579,374.228210,27.0,-0.982879,380.062108,-1.061822,-0.918300,-1.0,1.0,greenland,21,land
2,400323,-927905.065288,293849.709892,372.723389,27.0,-0.960258,378.503309,-1.015531,-0.899485,-1.0,1.0,greenland,21,land
3,400324,-927905.065288,293869.718204,370.800812,27.0,-0.920299,376.478685,-1.001954,-0.851485,-1.0,1.0,greenland,21,land
4,401523,-927925.080522,293789.684954,377.455170,27.0,-1.040214,383.679459,-1.101891,-0.958327,-1.0,1.0,greenland,21,land
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15501,817083,-934850.351469,280224.049044,876.999451,74.0,-0.420907,878.372902,-0.492269,-0.349391,-1.0,1.0,greenland,21,land
15502,817084,-934850.351469,280244.057357,876.509766,74.0,-0.423840,877.795194,-0.501019,-0.363567,-1.0,1.0,greenland,21,land
15503,817085,-934850.351469,280264.065669,875.957214,74.0,-0.423825,877.241198,-0.499648,-0.357873,-1.0,1.0,greenland,21,land


In [ ]:
cluster = LocalCluster(n_workers=4, threads_per_worker=2, memory_limit='8G')
client = cluster.get_client()
print(client.dashboard_link)

In [ ]:
directory

In [ ]:
get_summary_df(directory)

In [ ]:
df.compute()

In [ ]:
sec = xr.open_zarr(os.path.join(directory, 'sec.zarr'))
dem = xr.open_zarr(os.path.join(directory, 'stacked_coregd.zarr'))

In [ ]:
fig, axs = plt.subplots(subplot_kw={'projection':prj}, ncols=2, sharex=True, sharey=True)

dem_median.plot(ax=axs[0])
sec['sec'].sel(result='slope').plot(ax=axs[1], robust=True)

## fix centreline

In [ ]:
## fixing centrelines
centrelines = gpd.read_file('../data/streams_v3.geojson')
sites = glob('../data/id*')
for s in sites:
    id = int(os.path.basename(s).split('_')[0].replace('id',''))
    json_cl_path = glob('*.geojson', root_dir=s)
    json_cl = gpd.read_file(os.path.join(s, json_cl_path[0])).loc[0,'geometry']
    old_cntr = json_cl.centroid
    cl_new = centrelines.loc[id, 'geometry']
    new_cntr = cl_new.centroid
    assert cl_new.equals(json_cl), 'uh oh they are different'
    
    zarr_files = glob('*.zarr', root_dir=s)
    for zarr in zarr_files:
        with xr.open_zarr(os.path.join(s, zarr)) as ds:
            zarr_cl = wkt.loads(ds.attrs['centreline'])
            if zarr_cl.equals(cl_new):
                pass
                # print(f'all good in {zarr}')
                print(s, zarr, ds.dims)
            else:
                print('fuck')
                # # print(f'distance: {zarr_cl.centroid.distance(new_cntr):.2f}')
                # same_ends = zarr_cl.boundary.geoms[-1].equals(cl_new.boundary.geoms[-1])
                # if same_ends:
                #     print(f'the upglacier ends are the same: {same_ends} - overwriting attrs')
                #     ds.attrs.update({'centreline': cl_new.wkt})
                #     ds.to_zarr(os.path.join(s, zarr), mode='a')
                # else:
                #     print('not a clue what to do')

In [ ]:
fpath = '../data/id17_-167858x_-3142959y/sec.zarr/'
with xr.open_zarr(fpath) as ds:
    geom = wkt.loads(ds.attrs['centreline'])
    arr = np.zeros(ds['n'].shape)
    burned = rasterize(shapes=[(geom.buffer(200, cap_style=3), 1)],
                       fill=0,
                       out=arr,
                       transform=ds.rio.transform())
    merged = xr.merge([ds, xr.DataArray(data=burned,
                                        dims=['y','x'],
                                        coords={'y': ds.y,
                                                'x': ds.x}).rename('buffer_aoi')]
    )
    
    

In [ ]:
masked = xr.where(merged['buffer_aoi']==1, merged['sec'], np.nan)

In [ ]:
masked.sel(result='slope').plot()

In [ ]:
# buffer geometry, with square ends
buff_geom = cl.loc[0,'geometry'].buffer(200, cap_style=3)

# empty array of same x, y dim shape as merged
arr = np.zeros(merged['z_median'].shape)

# rasterize
burned = rasterize(shapes=[(buff_geom, 1)],
                fill=0,
                out=arr,
                transform=merged.rio.transform())

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection':prj})
ds['sec'].sel(result='slope').plot(ax=ax, robust=True, cmap='RdBu')
ax.plot(*wkt.loads(ds.attrs['centreline']).coords.xy, c='k', ls=':')

In [ ]:
df = da.concat([da.read_parquet(f) for f in glob('../data/**/elevation_sample.parquet')])
df = df.reset_index(drop=True)
df = df.compute()

# because this one in iceland is probably better described as lake terminating
# df.loc[df['id']=='29', 'lake_land'] = 'lake'

df['z_bins'] = pd.cut(df['z_median'], np.arange(-50,2000,100))
df['mid_bin'] = df['z_bins'].apply(lambda x: int(x.mid))
groups = df.groupby(['where', 'lake_land'])

In [ ]:
fg = sns.catplot(data=df.loc[df['where']=='greenland'],
                 x='mid_bin',
                 y='slope',
                 hue='lake_land',
                 col='id',
                 col_wrap=5,
                 fliersize=0.5,
                 kind='box',
                 )

fg.set(ylim=(-10,10))
fg.set_xticklabels(rotation=30)
fg.map(plt.axhline, y=0, c='k')

In [ ]:
sec = xr.open_zarr('../data/id29_1368245x_-2521726y/sec.zarr/')

In [ ]:
sampled_stack = ArcticDEM.sample_along_line('../data/id29_1368245x_-2521726y/stacked_coregd.zarr/')
sampled_sec = ArcticDEM.sample_along_line('../data/id29_1368245x_-2521726y/sec.zarr/')
sampled_sec = demote_coords_to_vars(sampled_sec, 'result', 'sec')

In [ ]:
sampled_stack.time.max()

In [ ]:
fig, ax = plt.subplots()
sampled_sec['slope'].plot(ax=ax)
ax.axhline(0, c='k')

In [ ]:
idx = (np.abs(sampled['median_after']) < 1) & (sampled['nmad_after'] < 2)

In [ ]:
colors = plt.get_cmap('viridis', len(sampled.time)).colors

fig = plt.figure(figsize=[6,6])
ax = fig.add_subplot(1,1,1)
ax.set_prop_cycle(cycler(color=colors))
_ = sampled['z'].sel(time=idx).plot(hue='time', ax=ax, add_legend=False, lw=0.5)
ax.set_ylim(50, 900)
ax.set_xlim(0, 8)

In [ ]:
fg = sns.catplot(data=df.loc[df['where']=='iceland'],
                 x='mid_bin',
                 y='slope',
                 hue='lake_land',
                 col='id',
                 col_wrap=5,
                 fliersize=0.5,
                 kind='box',
                 )

fg.set(ylim=(-10,10))
fg.set_xticklabels(rotation=30)
fg.map(plt.axhline, y=0, c='k')

In [ ]:
centrelines.reset_index().loc[centrelines['where']=='iceland'].explore(tiles = 'https://api.maptiler.com/tiles/satellite-v2/{z}/{x}/{y}.jpg?key=qXXaw6q8a36GQNC8fUDj',
                                                         attr='asdf')

In [ ]:
fig, axs = plt.subplot_mosaic([['greenland_lake', 'iceland_lake'],
                               ['greenland_land', 'iceland_land']],
                              sharex=True, sharey=True)

for where, lake_land in groups.groups:
    axes = f'{where}_{lake_land}'
    _tmp = groups.get_group((where, lake_land))
    
    _tmp = _tmp.loc[_tmp['slope'].abs() < 20]
    
    axs[axes].hexbin(_tmp['z_median'], _tmp['slope'], gridsize=(20,20), extent=(-50,1550,-10,10), cmap='Blues')
    
    # sns.kdeplot(_tmp, x='z_median', y='slope', ax=axs[axes])
    
    axs[axes].set_title(axes)


## results plotting

In [ ]:
# plot sec field and count of arctic DEMs for each site

sites = glob('../data/id*')
for site in sites:

    id = os.path.basename(site).split('_')[0][2:]
    sec = xr.open_zarr(site + '/sec.zarr')
    dem = xr.open_zarr(site + '/stacked_coregd.zarr')

    fig, axs = plt.subplots(ncols=2,
                            subplot_kw={'projection':prj})

    sec['sec'].sel(result='slope').plot(ax=axs[0],
                                        robust=True,
                                        cmap='RdBu',
                                        cbar_kwargs={'shrink': 0.6,
                                                     'label': '(m/yr)'})

    sec['n'].plot(ax=axs[1],
                  cbar_kwargs={'shrink':0.6,
                               'label': 'count'})

    for ax in axs:
        ax.plot(*wkt.loads(sec.attrs['centreline']).coords.xy, c='k', ls=':')

    axs[0].set_title(f'#{id} surface elevation change')
    axs[1].set_title(f'#{id}: count of ArcticDEMs')
    
    # fig.savefig(site + '/sec_result.png', dpi=600, bbox_inches='tight')
    plt.close(fig)

In [ ]:
sites = glob('../data/id*')

colors = {'lake': 'tab:blue',
          'land': 'tab:green'}

markers = {'greenland': 'o',
           'iceland': '+'}

fig, ax = plt.subplots()

dfs = []

for site in tqdm(sites):
    
    id = os.path.basename(site).split('_')[0][2:]
    
    cl_path = os.path.join(site, glob('*.geojson', root_dir=site)[0])
    cl = gpd.read_file(cl_path)
    
    where = cl.loc[0, 'where']
    lake_land = cl.loc[0, 'lake_land']
    
    sec_path = os.path.join(site, 'sec.zarr')
    dem_path = os.path.join(site, 'stacked_coregd.zarr')
    
    sampled_sec = ArcticDEM.sample_along_line(sec_path)
    sampled_sec = demote_coords_to_vars(sampled_sec, 'result', 'sec')
    
    sampled_dem = ArcticDEM.sample_along_line(dem_path, var='z')
    sampled_dem = sampled_dem['z'].median(dim='time').drop_vars('spatial_ref')
    
    sample = xr.merge([sampled_sec,
                       sampled_dem])
    
    df = sample.to_dataframe()
    
    df['where'] = where
    df['lake_land'] = lake_land
    df['id'] = id
    
    dfs.append(df)
    
    # sample.plot.scatter(x='z',
    #                     y='slope',
    #                     color=colors[lake_land],
    #                     marker=markers[where],
    #                     ax=ax
    #                     )

dataframe = da.concat(dfs)

In [ ]:
fig, axs = plt.subplot_mosaic([['greenland'],['iceland']], sharex=True, sharey=True)

directory = '../data/id5_-226289x_-2968847y/'
for directory in glob('../data/id*'):
    id = directory.split('id')[1].split('_')[0]
    
    centreline_filepath = glob('*.geojson', root_dir=directory)
    gdf = gpd.read_file(os.path.join(directory, centreline_filepath[0]))
    where = gdf.loc[0, 'where']
    
    sec_filepath = glob('sec.zarr', root_dir=directory)
    sec_filepath = os.path.join(directory, sec_filepath[0])
    z_filepath = glob('stacked*', root_dir=directory)
    z_filepath = os.path.join(directory, z_filepath[0])
    
    sec_along_line = sample_sec_along_line(sec_filepath)
    z_along_line = sample_sec_along_line(z_filepath, var='z')
    merged = xr.merge([sec_along_line.sel(result='slope').drop_vars('spatial_ref'),
                    z_along_line.median(dim='time')])

    merged[['z', 'sec']].to_dataframe().sort_values(by='z').plot(x='z', y='sec', label=id, ax=axs[where])
    
axs['greenland'].legend(fontsize=8, ncols=3, frameon=False, loc='upper right')
axs['iceland'].legend(fontsize=8, ncols=2, frameon=False)

axs['greenland'].set_title('greenland')
axs['iceland'].set_title('iceland')

axs['iceland'].set_xlabel('elevation (m)')
axs['iceland'].set_ylabel('sec (m/yr)')
axs['greenland'].set_ylabel('sec (m/yr)')

# fig.savefig('../results/elevation_change/sec_against_z.png', dpi=600, bbox_inches='tight')

In [ ]:
# add directory to centrelines geojson
centrelines = gpd.read_file('../data/streams_v3.geojson')
centrelines['directory'] = pd.Series()
for line in centrelines.itertuples():
    cntr = line.geometry.centroid
    centrelines.loc[line.Index, 'directory'] = f'../data/id{line.Index}_{cntr.x:.0f}x_{cntr.y:.0f}y'

In [ ]:
def plot_result(ds, id, ax, plot_err=False):
    
    ds['sec'].sel(result='slope').plot(ax=ax, label=id)
    if plot_err:
        ax.fill_between(
            x=ds.cumulative_distance,
            y1=ds['sec'].sel(result='high_slope').data,
            y2=ds['sec'].sel(result='low_slope').data,
            color='lightgrey')
    ax.axhline(0, c='grey', lw=0.5)

In [ ]:
import cartopy.feature as cfeature
from adjustText import adjust_text
from cycler import cycler
import seaborn as sns

In [ ]:
where = 'greenland'

colors = plt.get_cmap('tab20').colors

fig = plt.figure()

ax = fig.add_subplot(1,2,1)

ax.set_prop_cycle(cycler(color=colors))

axmap = fig.add_subplot(1,2,2, projection=prj)
axmap.add_feature(cfeature.COASTLINE, edgecolor='lightgrey')
centrelines.loc[centrelines['where']==where].plot(ax=axmap, color='k')
gl = axmap.gridlines(crs=ccrs.PlateCarree(),
                     draw_labels=True,
                     y_inline=False)
gl.rotate_labels=False

gl.bottom_labels=True
gl.left_labels=True
gl.top_labels=False
gl.right_labels=False
gl.ylabel_style = {'rotation': 90}


texts = []
for i, row in enumerate(centrelines.loc[centrelines['where'] == where].itertuples()):
    
    sec_file = glob('sec.zarr', root_dir=row.directory)
    assert len(sec_file) == 1, 'not enough / too many files'
    sec_file_path = os.path.join(row.directory, sec_file[0])
    
    sampled = ArcticDEM.sample_along_line(sec_file_path)
    
    plot_result(sampled, row.Index, ax)
    
    
    text = axmap.annotate(
        text=row.Index,
        xy=row.geometry.centroid.coords[0],
        c=colors[i],
        path_effects=[path_effects.withSimplePatchShadow(shadow_rgbFace='lightgrey',
                                                                 alpha=0.9,
                                                                 offset=(1,-1))])
    texts.append(text)
    
_ = adjust_text(texts)

# ax.legend(ncols=2, frameon=False)
ax.set_title(None)
ax.set_ylabel('surface elevation change (m/yr)')
ax.set_xlabel('distance from terminus (km)')

sns.despine(ax=ax)
axmap.set_axis_off()
plt.subplots_adjust(wspace=0.05)

ax.set_title(where)
ax.set_xlim(right=10.4)

fig.savefig(f'../results/elevation_change/{where}_sec.png', dpi=600, bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(ncols=2, subplot_kw={'projection':prj})

with LocalCluster() as c, c.get_client() as cli:
    with xr.open_dataset('../data/id9_484789x_-1406074y/sec.zarr/',
                         engine='zarr') as ds:
        # counts = ds['z'].isnull().sum(dim=['x', 'y'])
        ds['sec'].sel(result='slope').plot(robust=True, ax=axs[0], cbar_kwargs={'shrink':0.5})
        ds['n'].plot(ax=axs[1], cbar_kwargs={'shrink':0.5})
        cl = wkt.loads(ds.attrs['centreline'])    

for ax in axs:
    ax.plot(*cl.coords.xy, c='k')


In [ ]:
import seaborn as sns
def plot_stat(df, stat, ax):
    # df = get_meta_df(d)
    after = f'{stat}_after'
    before = f'{stat}_before'
    for row in df.iterrows():
        # print(row[0])
        ax.annotate("",
                    xy=(row[1]['to_reg_acqdate1'], row[1][after]),
                    xytext=(row[1]['to_reg_acqdate1'], row[1][before]),
                    arrowprops=dict(arrowstyle='->'))
    
    minx, maxx = df['to_reg_acqdate1'].agg(['min','max'])
    delta = pd.Timedelta('90d')
    ax.set_xlim(minx-delta, maxx+delta)
    ax.set_ylim(*df[[after, before]].melt()['value'].agg(['min','max']))
    
    ax.axhline(0, c='lightgrey', lw=0.5)
    ax.axvline(df.ref_acqdate1.unique()[0], ls=':', c='k')
    ax.set_ylabel(f'{stat} (m)')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='center')

def plot_coreg(df, ax):
    # df = get_meta_df(d)
    
    plot_df = df[['median_after','median_before','nmad_after','nmad_before']].melt()
    plot_df['when'] = plot_df['variable'].str.split('_').apply(lambda x: x[1])
    plot_df['variable'] = plot_df['variable'].str.split('_').apply(lambda x: x[0])
    sns.violinplot(data=plot_df,
                   x='variable',
                   y='value',
                   hue='when',
                   palette=sns.palettes.color_palette('colorblind')[-2:],
                   hue_order=['before','after'],
                   ax=ax)
    ax.set_ylabel('metres')
    ax.set_xlabel(None)
    ax.axhline(0, c='lightgrey', lw=0.5, zorder=0)
    sns.move_legend(ax, loc='best', title=None)

In [ ]:
directory = '../data/id8_607229x_-1847482y/'
file = glob('stack*', root_dir=directory)

In [ ]:
beach = False

if beach:
    with xr.open_dataset(os.path.join(directory, file[0]), engine='zarr') as ds:
        2+2
else:
    with rio.open_rasterio('../data/id0_-138892x_-3197992y/padded_SETSM_s2s041_W1W1_20180709_10200100727D6100_1020010076EA9B00_2m_lsf_seg1.tif') as ds:
        3+3
        

In [ ]:
xr.open_dataset(os.path.join(directory, file[0]), engine='zarr')['z'].rio.transform()

## velocity

In [ ]:
from dask.distributed import Client, LocalCluster
import pandas as pd
# import itslive
import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely import box
import matplotlib.pyplot as plt
# from matplotlib.collections import LineCollection
# from matplotlib.dates import date2num, DateFormatter, YearLocator
# import seaborn as sns
# import xrspatial as xrs
import numpy as np
import xarray as xr
from tqdm import tqdm
from IPython import display
import importlib
# import imagery
import utils
import velocity_helpers

In [ ]:
# _ = importlib.reload(utils)
# _ = importlib.reload(velocity_helpers)
# _ = importlib.reload(imagery)

In [ ]:
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

In [ ]:
## read in lines, and lazily get velocity cubes, centrelines, and robust trends
lines = gpd.read_file('../data/streams_v2.geojson').to_crs(3413)

In [ ]:
V = {}
failed = []
for row in tqdm(lines.sample(3).itertuples()):
    print(f'working on #{row.Index}')
    try:
        V[row.Index] = velocity_helpers.CentreLiner(
            geo=row.geometry,
            buff_dist=500,
            index=row.Index,
            filter_cube=False,
            get_robust_trend=True,
            get_annual_median=False,
            get_rgb=False,
            )
    except Exception as e:
        failed.append((row.Index, e))
        print(f'#{row.Index} did not work because\n{e}')
        continue

## plotting

In [ ]:
fig, axs = plt.subplots(figsize=[15,8],
                        nrows=2,
                        ncols=4,
                        sharex=True,
                        sharey=True)

for k_v, ax in zip(V.items(), axs.flat):
    k, v = k_v
    velocity_helpers.Plotters.rolling_median(v,
                                             ax=ax,
                                         **{'var': 'v',
                                            'ddt_range': ('335d','395d'),
                                            'ddt_bars': False,
                                            'col': 'cumul_dist',
                                            'window': '90d',
                                            'vals':[500, 1_000, 3_000, 5_000]
                                            })
    _point = utils.shapely_reprojector(v.point, 3413, 4326)
    
    ax.set_title(f'#{k}: {_point.y:.2f}N {_point.x:.2f}W\n{lines.loc[k,"name"]}')
    ax.minorticks_off()

# axs.flat[-2].remove()
axs.flat[-1].remove()

now = pd.Timestamp.now().strftime('%y%m%d_%H%M')+ '_'
directory = '../../GitHub/diary/journal_figures'
filename = 'iceland_outlets_annual_v.png'
fig.savefig(directory + now + filename, bbox_inches = 'tight')

In [ ]:
def centreline_theil_slopes(df, **kwargs):
    
    _var = kwargs.get('var', 'v')
    _x = kwargs.get('x', 'mid_date')
    _date_range = kwargs.get('date_range', (pd.Timestamp('1900-01-01'),
                                            pd.Timestamp.now()))
    _ddt_range = kwargs.get('ddt_range', ('0d', '30d'))
    _mad = kwargs.get('mad', 3)
    
    _df = df.loc[df[_x].between(*_date_range)]    

In [ ]:
idx = ((ds['date_dt'] > pd.Timedelta('335d')) & ((ds['date_dt'] <= pd.Timedelta('395d')))).compute()

from scipy.stats.mstats import theilslopes
from scipy.stats import linregress

x = (ds.mid_date[idx] - ds.mid_date[idx].min()) / pd.Timedelta('365.25d')
y = ds['v'][idx,50,50].compute()

nan_idx = y.isnull()

x = x[~nan_idx]
y= y[~nan_idx]

theil_result = theilslopes(y=y, x=x)
lr_result = linregress(x=x, y=y)


X = np.arange(0,30)
theil_fit = (X * theil_result.slope) + theil_result.intercept
lr_fit = (X * lr_result.slope) + lr_result.intercept

fig, ax = plt.subplots()
ax.scatter(x,y)

ax.plot(X, theil_fit, c='r', label='theilslopes')
ax.plot(X, lr_fit, c='g', label='lr')
ax.legend()

In [ ]:
fig, axs = plt.subplots(figsize=[15,8], nrows=5, ncols=3)
# get_velocity.Plotters.date_dt_bars(V[13],
#                                    ('1d','15d'),
#                                    ax=ax,
#                                    **{'vals':[5_000,1_000]})

get_velocity.Plotters.rolling_median(V[13],
                                     ('335d','395d'),
                                     ax=ax,
                                     ddt_bars=False,
                                     **{'var': 'v',
                                        'col': 'cumul_dist',
                                        'window': '180d',
                                        'vals':[1_000, 3_000, 5_000, 10_000]})

- look at underlying dodgy velocity images
TODO look at GRIMpP and MEASUREs

TODO
- PN thinks that relative change is sufficient to *control* for variations in glacier geometry (read: width/size).
- most interested in *relative* change of velocity over final few kilometers.
    - i.e. 

- it all comes from the same raw processed v data

## imagery

In [ ]:
study_sites = gpd.read_file('../data/potential_study_sites_v1.geojson')
study_sites = study_sites.loc[study_sites['notes'].isin(['yes', 'maybe'])]
study_sites.reset_index(drop=True, inplace=True)
study_sites_5k = study_sites.buffer(5_000).to_crs(4326)

In [ ]:
items = imagery.get_annual_median_mosaic(
    study_sites.loc[0,'geometry'],
    buffer_dist=15_000,
    src_crs=study_sites.crs,
    target_crs=4326,
    timeperiod='2023-01-01/2024-01-01',
    months=[8,9],
    )

In [ ]:
(xrs.multispectral.true_color(
    *items.squeeze()
    .transpose('band', 'y', 'x'))
 .plot.imshow(rgb='band')
)

In [ ]:
import stackstac
stackstac.stack(items, assets=['B04', 'B03', 'B02'])